# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [2]:
#importing necessary libraries
import pandas as pd
import sqlite3

#Establishing connection to database
conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",end="")
for table in tables: print(table[0],end=", ")
print("\b\b")

ALL TABLES: BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA, 


# FIRST PHASE OF TOOL

In [3]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

Enter your zipcode: 07040


## Step 3, filtering formulary_ids by pdp_region_code and contract_id

In [4]:
F_SUBSET = pd.read_sql("select * from PLAN_INFO\
                        where PDP_REGION_CODE is '"+str(user_pdp_region_code)+"'\
                        and CONTRACT_ID like 'S%'",conn)\
           [['CONTRACT_ID','PLAN_ID','SEGMENT_ID','FORMULARY_ID','PREMIUM','DEDUCTIBLE','PDP_REGION_CODE']]
F_SUBSET

,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,PDP_REGION_CODE
0,S0522,4,0,18041,30.4,405.0,4
1,S4802,78,0,18443,36.9,405.0,4
2,S4802,101,0,18439,79.5,0.0,4
3,S5601,8,0,18419,34.3,0.0,4
4,S5601,9,0,18420,84.6,0.0,4
5,S5617,18,0,18082,74.8,405.0,4
6,S5617,249,0,18083,52.9,0.0,4
7,S5660,106,0,18154,34.0,405.0,4
8,S5660,207,0,18155,91.7,350.0,4
9,S5660,220,0,18152,22.6,405.0,4


## Step 4, asking user for drugs and strengths

In [5]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):[]\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
user_generics

Enter how many drugs you wish to search: 3
Enter a generic drug name: Allopurinol
Enter a desired strength: 300
Enter a generic drug name: metformin
Enter a desired strength: 500
Enter a generic drug name: metoprolol succinate
Enter a desired strength: 50


{('ALLOPURINOL', '300'): [],
 ('METFORMIN', '500'): [],
 ('METOPROLOL SUCCINATE', '50'): []}

## STEP 5

In [6]:
for drug,strength in user_generics:
    user_generics[(drug,strength)] = \
    pd.read_sql("select PLAN_INFO.CONTRACT_ID, PLAN_INFO.PLAN_ID, FORMULARY_DATA.* \
from (PLAN_INFO join FORMULARY_DATA \
      on instr(FORMULARY_DATA.FORMULARY_ID, PLAN_INFO.FORMULARY_ID) > 0)\
      where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' \
      and PLAN_INFO.CONTRACT_ID like 'S%' \
      and FORMULARY_DATA.RXCUI in (select RXCUI_DATA.RXCUI from RXCUI_DATA\
    where upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and\
    RXCUI_DATA.STRENGTH like '%"+strength+" %')",conn)
#     print ("DRUG: %s\nSTRENGTH: %s\nDATA: \n%s\n%s"%\
#            (drug,strength,"-"*80,user_generics[(drug,strength)]))

## STEP 6

In [8]:
for drug,strength in user_generics:
    user_generics[(drug,strength)] = \
pd.read_sql("select distinct E.FORMULARY_ID, E.RXCUI, BENEFICIARY_COST.CONTRACT_ID, \
BENEFICIARY_COST.PLAN_ID, E.PLAN_NAME, E.PRIOR_AUTHORIZATION_YN, \
BENEFICIARY_COST.TIER, BENEFICIARY_COST.DAYS_SUPPLY, \
BENEFICIARY_COST.COST_AMT_PREF, E.PREMIUM, E.DEDUCTIBLE from \
((select PLAN_INFO.CONTRACT_ID, PLAN_INFO.PLAN_NAME, PLAN_INFO.PLAN_ID, \
PLAN_INFO.PREMIUM, PLAN_INFO.DEDUCTIBLE, FORMULARY_DATA.* \
from (PLAN_INFO join FORMULARY_DATA \
      on instr(FORMULARY_DATA.FORMULARY_ID, PLAN_INFO.FORMULARY_ID) > 0)\
      where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' \
      and PLAN_INFO.CONTRACT_ID like 'S%' \
      and FORMULARY_DATA.RXCUI in (select RXCUI_DATA.RXCUI from RXCUI_DATA\
    where upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and\
    RXCUI_DATA.STRENGTH like '%"+strength+" %')) as E) join BENEFICIARY_COST on \
    (BENEFICIARY_COST.CONTRACT_ID = E.CONTRACT_ID and \
     BENEFICIARY_COST.PLAN_ID = E.PLAN_ID and \
     cast(BENEFICIARY_COST.TIER as integer) = cast(E.TIER_LEVEL_VALUE as integer)) \
where (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
       cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
      BENEFICIARY_COST.DAYS_SUPPLY = 1 and\
      cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1",conn)

In [9]:
#EXAMPLE FOR METFORMIN
user_generics[('METFORMIN','500')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TIER,DAYS_SUPPLY,COST_AMT_PREF,PREMIUM,DEDUCTIBLE
0,00018039,861740,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0
1,00018039,861736,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0
2,00018039,1664321,S5820,3,AARP MedicareRx Preferred (PDP),N,3,1,37.0,86.6,0.0
3,00018039,1664325,S5820,3,AARP MedicareRx Preferred (PDP),N,3,1,37.0,86.6,0.0
4,00018039,861007,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0
5,00018039,1545166,S5820,3,AARP MedicareRx Preferred (PDP),N,3,1,37.0,86.6,0.0
6,00018039,1545159,S5820,3,AARP MedicareRx Preferred (PDP),N,3,1,37.0,86.6,0.0
7,00018039,1811011,S5820,3,AARP MedicareRx Preferred (PDP),N,3,1,37.0,86.6,0.0
8,00018039,1811003,S5820,3,AARP MedicareRx Preferred (PDP),N,3,1,37.0,86.6,0.0
9,00018039,860975,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0


In [10]:
#EXAMPLE FOR ALLOPURINOL
user_generics[('ALLOPURINOL','300')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TIER,DAYS_SUPPLY,COST_AMT_PREF,PREMIUM,DEDUCTIBLE
0,00018039,197320,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0
1,00018040,197320,S5921,349,AARP MedicareRx Saver Plus (PDP),N,1,1,1.0,33.2,405.0
2,00018041,197320,S0522,4,Symphonix Value Rx (PDP),N,1,1,1.0,30.4,405.0
3,00018043,197320,S5921,386,AARP MedicareRx Walgreens (PDP),N,1,1,0.0,26.8,405.0
4,00018056,197320,S5810,38,Aetna Medicare Rx Saver (PDP),N,1,1,1.0,34.9,325.0
5,00018057,197320,S5810,278,Aetna Medicare Rx Select (PDP),N,1,1,0.0,19.7,405.0
6,00018059,197320,S5768,127,First Health Part D Value Plus (PDP),N,1,1,1.0,56.2,0.0
7,00018082,197320,S5617,18,Cigna-HealthSpring Rx Secure (PDP),N,1,1,1.0,74.8,405.0
8,00018083,197320,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),N,1,1,4.0,52.9,0.0
9,00018130,197320,S5993,1,Horizon Medicare Blue Rx Standard (PDP),N,2,1,8.0,47.7,405.0


In [11]:
user_generics[('METOPROLOL SUCCINATE','50')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TIER,DAYS_SUPPLY,COST_AMT_PREF,PREMIUM,DEDUCTIBLE
0,00018039,866436,S5820,3,AARP MedicareRx Preferred (PDP),N,2,1,10.0,86.6,0.0
1,00018040,866436,S5921,349,AARP MedicareRx Saver Plus (PDP),N,1,1,1.0,33.2,405.0
2,00018041,866436,S0522,4,Symphonix Value Rx (PDP),N,1,1,1.0,30.4,405.0
3,00018043,866436,S5921,386,AARP MedicareRx Walgreens (PDP),N,1,1,0.0,26.8,405.0
4,00018056,866436,S5810,38,Aetna Medicare Rx Saver (PDP),N,2,1,2.0,34.9,325.0
5,00018057,866436,S5810,278,Aetna Medicare Rx Select (PDP),N,2,1,3.0,19.7,405.0
6,00018059,866436,S5768,127,First Health Part D Value Plus (PDP),N,2,1,2.0,56.2,0.0
7,00018082,866436,S5617,18,Cigna-HealthSpring Rx Secure (PDP),N,1,1,1.0,74.8,405.0
8,00018083,866436,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),N,1,1,4.0,52.9,0.0
9,00018130,866436,S5993,1,Horizon Medicare Blue Rx Standard (PDP),N,2,1,8.0,47.7,405.0


## STEP 7, 8A, 8B, 8C

In [12]:
totals = {(drug,strength):None for drug,strength in user_generics}

In [13]:
for key in totals:
    totals[key] = user_generics[key][['FORMULARY_ID','RXCUI','CONTRACT_ID','PLAN_ID',\
                                     'PLAN_NAME','PRIOR_AUTHORIZATION_YN']]
    totals[key]['TOTAL_ANNUAL_COST'] = user_generics[key]['DEDUCTIBLE'] + \
                                       user_generics[key]['PREMIUM']*12 + \
                                       user_generics[key]['COST_AMT_PREF']*12
    totals[key] = totals[key].sort_values('TOTAL_ANNUAL_COST')[:5] #CHEAPEST 5

/Users/khayyamsaleem/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
#EXAMPLE WITH LISINOPRIL
totals[('METOPROLOL SUCCINATE','50')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TOTAL_ANNUAL_COST
18,00018419,866436,S5601,8,SilverScript Choice (PDP),N,579.6
5,00018057,866436,S5810,278,Aetna Medicare Rx Select (PDP),N,677.4
8,00018083,866436,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),N,682.8
15,00018255,866436,S5884,150,Humana Walmart Rx Plan (PDP),N,697.8
6,00018059,866436,S5768,127,First Health Part D Value Plus (PDP),N,698.4


In [15]:
totals[('METFORMIN','500')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TOTAL_ANNUAL_COST
158,00018419,861007,S5601,8,SilverScript Choice (PDP),N,447.6
163,00018419,860975,S5601,8,SilverScript Choice (PDP),N,447.6
153,00018419,861740,S5601,8,SilverScript Choice (PDP),N,579.6
156,00018419,861736,S5601,8,SilverScript Choice (PDP),N,579.6
51,00018057,861007,S5810,278,Aetna Medicare Rx Select (PDP),N,641.4


In [16]:
totals[('ALLOPURINOL','300')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TOTAL_ANNUAL_COST
18,00018419,197320,S5601,8,SilverScript Choice (PDP),N,579.6
5,00018057,197320,S5810,278,Aetna Medicare Rx Select (PDP),N,641.4
15,00018255,197320,S5884,150,Humana Walmart Rx Plan (PDP),N,661.8
8,00018083,197320,S5617,249,Cigna-HealthSpring Rx Secure-Extra (PDP),N,682.8
6,00018059,197320,S5768,127,First Health Part D Value Plus (PDP),N,686.4


## TODO: SECOND PHASE OF TOOL

In [ ]:
#CODE FOR STEP 5 AND 6 WITH NDC's INSTEAD OF RxCUI's, FAILED ATTEMPT


#getting NDCs for each drug,strength combo the user offered in step 4
# for drug,strength in user_generics:
#     user_generics[(drug,strength)] = pd.read_sql("select PRODUCTNDC from NDC_DATA\
#                  where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
#                  and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+")",conn)\
#     .applymap(lambda s: "0"*(11-(len(s)-1))+s.replace("-",""))

# for drug,strength in user_generics:
#     print("DRUG: %s, STRENGTH: %s\n%s" % (drug,strength,user_generics[drug,strength]))

# drug = 'LISINOPRIL'
# strength = 10
# pd.read_sql("select * from FORMULARY_DATA\
#  where NDC in (select PRODUCTNDC from NDC_DATA\
#                  where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
#                  and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+"))",conn)
    

# pd.read_sql("select * from FORMULARY_DATA",conn)

# CLEAN_NDC_DATA_TABLE = pd.read_sql("select PRODUCTNDC from NDC_DATA",conn).\
#applymap(lambda s: s.replace("-","")).iloc[1:]
# def clean_ndc(s):
#     if len(s) in [11,10,9]:
        
# CLEAN_NDC_FD_TABLE = pd.read_sql("select NDC from FORMULARY_DATA",conn).applymap()

#RXCUI REWORK
# pd.read_sql("select distinct RXN_DOSE_FORM from RXCUI_DATA",conn)